In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [8]:
gc = GraphCreator("Decision tree", include_see_also=False)
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 257 


['Probability', 'Algorithm', 'Decision support system', 'Goal', 'Decision analysis', 'Utility', 'Operations research', 'Decision tree learning', 'Causal model', 'Tree (graph theory)'] 


['Behavior tree (artificial intelligence, robotics and control)', 'Boosting (machine learning)', 'Decision cycle', 'Decision list', 'Decision table', 'Decision tree model', 'Design rationale', 'DRAKON', 'Markov chain', 'Random forest', 'Odds algorithm', 'Topological combinatorics', 'Truth table']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [9]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [12]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [13]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Hysteria Project,Hysteria Project 2,7
1,Hysteria Project 2,Hysteria Project,7
2,Deep learning,Glossary of artificial intelligence,5
3,Mind map,Concept map,5
4,Glossary of artificial intelligence,Deep learning,5


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [14]:
features_df = gc.get_features_df(rank=False)

In [ ]:
features_df.sort_values("degree", ascending=False)

## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [23]:
gc.rank_similarity()

# print(gc.see_also_articles)

gc.features_df[[
    "node", 
    "category_matches_with_source", 
    "primary_link", 
    "shared_neighbors_with_entry_score",
    "shortest_path_length_from_entry",
    "shortest_path_length_to_entry",
    "adjusted_reciprocity",
    "centrality",
    "jaccard_similarity",
    "similarity_rank", 
]].sort_values(["similarity_rank"], ascending=False).reset_index().drop("index", axis=1)
# gc.features_df[gc.features_df.category_matches_with_source == 1]

,node,category_matches_with_source,primary_link,shared_neighbors_with_entry_score,shortest_path_length_from_entry,shortest_path_length_to_entry,adjusted_reciprocity,centrality,jaccard_similarity,similarity_rank
0,Decision tree,1,0,1.000000,0.0,0.0,68.0,8.854657e-02,1.000000,inf
1,Decision tree learning,1,1,0.067442,2.0,2.0,180.0,1.264242e-01,0.052469,1.033721
2,Decision analysis,1,1,0.065359,2.0,2.0,28.0,1.203585e-02,0.039130,1.032680
3,Tree (graph theory),0,1,0.018933,1.0,1.0,56.0,1.449512e-02,0.010460,1.018933
4,Decision support system,0,1,0.036304,1.0,2.0,132.0,7.633322e-02,0.022869,0.690869
5,Causal model,0,1,0.026667,1.0,2.0,4.0,5.467511e-03,0.000000,0.684444
6,Operations research,0,1,0.014886,1.0,2.0,338.0,1.469221e-01,0.007051,0.676590
7,Probability,0,1,0.013374,1.0,2.0,242.0,1.199587e-01,0.010423,0.675583
8,Algorithm,0,1,0.011272,1.0,2.0,206.0,1.692748e-01,0.008291,0.674181
9,Utility,0,1,0.007506,1.0,2.0,472.0,4.909813e-02,0.003289,0.671671


# Evaluation

In [24]:
evaluate_metrics(gc.features_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry"], 
                 targets=gc.see_also_articles)

Metric Score,score,max score possible,difference,total targets,% targets in top 1%,% targets in top 5%,% targets in top 10%,% targets in top 20%
similarity_rank,0.908321,0.999691,0.091370,13.0,0.461538,0.769231,1.000000,1.0
centrality,0.849644,0.999691,0.150046,13.0,0.538462,0.846154,0.923077,1.0
adjusted_reciprocity,0.852476,0.999691,0.147215,13.0,0.769231,0.923077,0.923077,1.0
page_rank,0.947201,0.999691,0.052490,13.0,0.769231,0.923077,1.000000,1.0
shortest_path_length_from_entry,0.502962,0.999691,0.496728,13.0,0.000000,0.000000,0.000000,0.0


In [25]:
df = gc.features_df.sort_values("page_rank", ascending=False).reset_index().drop("index", axis=1)
see_also_indeces = []
for node in gc.see_also_articles:
    article = df[df.node == node]
    
    see_also_indeces.append((node, article.index[0]))
    
see_also_indeces

[('Behavior tree (artificial intelligence, robotics and control)', 1150),
 ('Boosting (machine learning)', 75),
 ('Decision cycle', 641),
 ('Decision list', 2219),
 ('Decision table', 132),
 ('Decision tree model', 200),
 ('Design rationale', 67),
 ('DRAKON', 92),
 ('Markov chain', 29),
 ('Random forest', 63),
 ('Odds algorithm', 145),
 ('Topological combinatorics', 99),
 ('Truth table', 24)]

In [ ]:
len(gc.graph.nodes)